In [13]:
import re

from pathlib import Path
from time import sleep
from typing import Dict, List

import pandas as pd
import PyPDF2
import requests

from selenium import webdriver
from selenium.webdriver.common.by import By
from tqdm import tqdm

In [14]:
URLS = [
    'https://qualifications.pearson.com/en/qualifications/edexcel-a-levels/business-2015.coursematerials.html#%2FfilterQuery=category:Pearson-UK:Category%2FExam-materials&filterQuery=category:Pearson-UK:Document-Type%2FQuestion-paper&filterQuery=category:Pearson-UK:Unit%2FPaper-1',
    'https://qualifications.pearson.com/en/qualifications/edexcel-international-advanced-levels/business-2018.coursematerials.html#filterQuery=category:Pearson-UK:Category%2FExam-materials&filterQuery=category:Pearson-UK:Document-Type%2FQuestion-paper&filterQuery=category:Pearson-UK:Unit%2FUnit-1'
]

Данные URL открываются только под VPN, для работы парсера он тоже нужен

In [17]:
def parse_web_page(url: str) -> None:
    """
        Извлечение pdf файлов по данному url
    """
    with webdriver.Chrome() as browser:
        browser.get(url)
        browser.maximize_window() 
        sleep(10) # На случай долгой прогрузки

        browser.find_element(By.ID, 'onetrust-accept-btn-handler').click() # Согласие на куки
        expand_all = browser.find_element(By.CSS_SELECTOR, 'a[ng-click="expandFiltersNew()"]').click() # Раскрытие видимости всех pdf файлов
        files = browser.find_elements(By.CLASS_NAME, 'doc-title') # Поиск всех pdf и сохранение в список
        padlocks = browser.find_elements(By.CLASS_NAME, 'padlock' ) # Парсинг состояния доступности для скачивания
        sleep(2)

        cnt = 0 # Почему-то находится в два раза больше файлов, необходимо брать каждый второй
        for file, padlock in zip(files, padlocks):
            if cnt % 2 == 0:
                secure = padlock.get_attribute('src') # Если атрибут src пустой, значит pdf доступен для открытия
                if secure == None:
                    main_window = browser.current_window_handle # Запоминаем текущую вкладку (сам сайт)
                    file.click() # Открываем ссылку на pdf
                    pdf_url = ''

                    for handle in browser.window_handles:
                        sleep(0.5)
                        if handle != main_window: # Переход на вкладку, в которой отрывается pdf
                            browser.switch_to.window(handle)
                            sleep(1)
                            pdf_url = browser.current_url
                            browser.close()

                    browser.switch_to.window(main_window) # Возвращаемся на сайт
                    response = requests.get(pdf_url).content # Скачивание контента
                    our_file = open(f"{file.text}.pdf", 'wb')
                    our_file.write(response)
            cnt += 1
            sleep(1.5)
        sleep(5)


In [18]:
for url in URLS:
    parse_web_page(url)
    

In [19]:
QUESTION_PATTERN = re.compile(r'\(([a-zA-Z])\)(.*?)\(([0-9]+)\)')


In [20]:
def extract_questions_from_page(text: str) -> Dict[str, int]:
    """
        Извлечение вопросов экзамена с конкретной страницы документа
        Возвращает словарь формата {'вопрос': количество баллов за него (int)}
    """
    questions = {}
    for question in re.findall(QUESTION_PATTERN, text.replace('\n', '')): # При парсинге pdf образуется много лишних переносов строк, их заменяем на ''
        questions[question[1].strip()] = int(question[2].strip())
    return questions

In [21]:
def extract_questions_from_file(file_name: str) -> pd.DataFrame:
    """
        Отктрытие файла по его названию и извлечение вопросов из него
        Возвращает датафрейм, содержащий название файла, вопрос, баллы за него и номер страницы, на которой он располагался
    """
    df = pd.DataFrame()
    with open(f'documents/{file_name}', 'rb') as file:
        pdf_reader = PyPDF2.PdfReader(file)
        for i_page in range(len(pdf_reader.pages)):
            questions = extract_questions_from_page(pdf_reader.pages[i_page].extract_text())
            tmp = pd.DataFrame({'question': questions.keys(), 'score': questions.values()})
            tmp['page'] = i_page + 1
            df = pd.concat([df, tmp])

    df['file'] = file_name
    df.score = df.score.astype('int')

    return df.reset_index(drop=True)


In [22]:
path = Path('documents')
file_names = [f.name for f in path.iterdir() if f.is_file()]


In [23]:
questions = pd.DataFrame()

for file_name in tqdm(file_names):
    questions = pd.concat([questions, extract_questions_from_file(file_name)])

100%|██████████| 26/26 [00:06<00:00,  4.06it/s]


In [24]:
questions = questions.reset_index(drop=True)

In [25]:
questions

,question,score,page,file
0,"Define the term ‘brand’ . (Extract A, line 9)",2,2,Question Paper - Unit 1 (WBS11) - January 2021...
1,Explain one r isk Jack Ma may have taken when ...,4,2,Question Paper - Unit 1 (WBS11) - January 2021...
2,Analyse t wo factors that may have increased d...,6,3,Question Paper - Unit 1 (WBS11) - January 2021...
3,Discuss if profit maximisa tion is the main bu...,8,4,Question Paper - Unit 1 (WBS11) - January 2021...
4,Assess the advantages of a paternalistic style...,10,5,Question Paper - Unit 1 (WBS11) - January 2021...
...,...,...,...,...
265,What is meant by a franchise?,2,9,Question paper - AS Level Paper 1 - June 2017.pdf
266,Explain how a flexible workforce might benefit...,4,9,Question paper - AS Level Paper 1 - June 2017.pdf
267,Construct a supply and demand diagram to illus...,4,10,Question paper - AS Level Paper 1 - June 2017.pdf
268,Assess two benefits to SSP of operating franc...,8,11,Question paper - AS Level Paper 1 - June 2017.pdf


In [26]:
(
        questions
    .groupby('file', as_index=False)
    .agg(
        question_cnt=('score', 'count'),
        score_sum=('score', 'sum')
    )
).to_excel('summary.xlsx', index=False)

In [27]:
questions[['file', 'page', 'question', 'score']].to_excel('questions.xlsx', index=False)